In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

# Titanic dataset

In [75]:
import os
TITANIC_PATH = os.path.join("datasets", "titanic")

In [76]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [77]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [78]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [79]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [80]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

In [81]:
y_train = train_data['Survived'].values
print(y_train.shape)
print(train_data.shape)

(891,)
(891, 12)


Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

Atrybut **Embarked** mówi nam, gdzie pasażer zaokrętował: C = Cherbourg, Q = Queenstown, S = Southampton.

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [82]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age", "Pclass", "SibSp", "Parch", "Fare"])),
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [84]:
num_pipeline.fit_transform(train_data)

array([[-0.56573646,  0.82737724,  0.43279337, -0.47367361, -0.50244517],
       [ 0.66386103, -1.56610693,  0.43279337, -0.47367361,  0.78684529],
       [-0.25833709,  0.82737724, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [-0.1046374 ,  0.82737724,  0.43279337,  2.00893337, -0.17626324],
       [-0.25833709, -1.56610693, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.20276197,  0.82737724, -0.4745452 , -0.47367361, -0.49237783]])

Będziemy także potrzebować imputera do kategorycznych kolumn  napisowych (zwykły Imputer nie działa na tych kolumnach):

In [85]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować **pipeline** dla atrybutów kategorycznych:

In [86]:
# from future_encoders import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

In [87]:
cat_pipeline.fit_transform(train_data)

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

Na koniec połączmy powyższe podejścia:

In [88]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline)
    ])

Teraz mamy fajny **pipeline** przetwarzania wstępnego, który pobiera dane wejściowe i zwraca dane wyjściowe złożone z liczb, które możemy podać do dowolnego modelu uczenia maszynowego.

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM liniowego
* SVM z jądrem rbf
* SVM z jądrem poly
* Regresji logistycznej
* RFR

In [89]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [20]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', SVC(kernel='linear'))])


param_grid = {
            'classifier__C': [0, 5, 7, 10]
}

svm_linear = GridSearchCV(pipe, param_grid, cv=kfold)

svm_linear.fit(train_data, y_train)
svm_linear.best_params_

/Users/jakubstepien/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jakubstepien/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jakubstepien/miniforge3/envs/env_tf/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/jakubstepien/miniforge3/envs/env_tf/lib/python3.9/site-package

{'classifier__C': 5}

In [24]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', SVC(kernel='rbf'))])


param_grid = {
            'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

svm_rbf = GridSearchCV(pipe, param_grid, cv=kfold)

svm_rbf.fit(train_data, y_train)
svm_rbf.best_params_

{'classifier__C': 100, 'classifier__gamma': 0.01}

In [25]:
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', SVC(kernel='poly'))])


param_grid = {
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 300, 500]
}

svm_poly = GridSearchCV(pipe, param_grid, cv=kfold)

svm_poly.fit(train_data, y_train)
svm_poly.best_params_

{'classifier__C': 1}

In [96]:
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', LogisticRegression())])

param_grid = {
            'classifier__C': [0.001, 0.01, 0.1,0.5, 1, 10, 100]
}
logistic_regression = GridSearchCV(pipe, param_grid, cv=kfold)
logistic_regression.fit(train_data, y_train)
logistic_regression.best_params_

{'classifier__C': 0.1}

In [27]:
from sklearn.ensemble import RandomForestRegressor

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', RandomForestRegressor())
])

param_grid={"classifier__max_depth": [5,10,15,20,25],
                               "classifier__max_features": [1,2,3,4,5,6,7,8,9],
                              "classifier__n_estimators": [10,20,30,50,100,500]}

rfr = GridSearchCV(pipe, param_grid, cv=kfold)
rfr.fit(train_data, y_train)
rfr.best_params_

{'classifier__max_depth': 5,
 'classifier__max_features': 9,
 'classifier__n_estimators': 20}

In [70]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

voting_clf = VotingClassifier(
    estimators=[('svm_linear', SVC(kernel='linear')), 
                ('svm_rbf', SVC(kernel='rbf')), 
                ('svm_poly', SVC(kernel='poly')), 
                ('rfr', RandomForestClassifier())]
)

voting = Pipeline([
    ('preprocessing', preprocess_pipeline),
    ('classifier', voting_clf)
])

voting.fit(train_data, y_train)

Pipeline(steps=[('preprocessing',
                 FeatureUnion(transformer_list=[('num_pipeline',
                                                 Pipeline(steps=[('select_numeric',
                                                                  DataFrameSelector(attribute_names=['Age',
                                                                                                     'Pclass',
                                                                                                     'SibSp',
                                                                                                     'Parch',
                                                                                                     'Fare'])),
                                                                 ('imputer',
                                                                  SimpleImputer(strategy='median')),
                                                                 ('scaler',
            

In [22]:
test_passenger_id = test_data["PassengerId"]
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [97]:
models = []
models.append(('SVM_linear', svm_linear.best_estimator_))
models.append(('SVM_polynomial', svm_poly.best_estimator_))
models.append(('SVM_rbf', svm_rbf.best_estimator_))
models.append(('Logistic_regression', logistic_regression.best_estimator_))
models.append(('RFR', rfr.best_estimator_))
models.append(('Voting', voting))

for name, model in models:
    result = model.predict(test_data)
    
    if name == 'RFR':
        result = [1 if x > 0.5 else 0 for x in result]
    
    result_dict = {
        'PassengerId': test_passenger_id,
        'Survived': result
    }
    
    result_df = pd.DataFrame(result_dict, columns=['PassengerId', 'Survived'])
    file_name = "results/" + name + '.csv'
    result_df.to_csv(file_name, index=False)